<a href="https://colab.research.google.com/github/yi-ye-zhi-qiu/kitchensoundscapes/blob/main/lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
import numpy as np
import os
from music21 import converter, instrument, note, chord, stream
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.layers import BatchNormalization as BatchNorm
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint
from keras.callbacks import *
from keras.optimizers import RMSprop 

# Get Notes

In [17]:
def get_notes():
    path='/content/drive/My Drive/notebooks/music/training_songs'
    notes = []

    for filename in os.listdir(path):
          file = path + "/" + filename
          print("Parsing", file)
        
          midi = converter.parse(file)
          need_parse = midi.flat.notes

          for i in need_parse:
            if isinstance(i, note.Note):
                notes.append(str(i.pitch))
            elif isinstance(i, chord.Chord):
                notes.append('.'.join(str(n) for n in i.normalOrder))
    return notes
n = get_notes()


Parsing /content/drive/My Drive/notebooks/music/training_songs/breadknife.mid


# Train LSTM

In [18]:
def prepare_sequences(notes, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = 5
    print('Sequence length (window) used', sequence_length)

    pitchnames = sorted(set(item for item in notes))
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
    print('Dict created note->int', note_to_int)

    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    network_input = network_input / float(n_vocab)
    print('Glance @ input', network_input[1:10])

    network_output = np_utils.to_categorical(network_output)
    print('Glance @ output', network_output[1:10])

    return (network_input, network_output)

In [19]:
def create_network(network_input, n_vocab):
    """ create the structure of the neural network """
    print('Creating structure of LSTM')
    model = Sequential()
    model.add(LSTM(
        100,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        recurrent_dropout=0.7,
        return_sequences=True
    ))
    model.add(LSTM(100))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('sigmoid'))
    model.compile(loss='categorical_crossentropy', optimizer=RMSprop(learning_rate=0.00000013))

    return model


In [20]:
def train(model, network_input, network_output):
    """ train the neural network """
    print('Training model...')
    filename = "/content/drive/My Drive/notebooks/music/weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
    checkpoint = ModelCheckpoint(
        filename,
        monitor='loss',
        verbose=0,
        save_best_only=True
    )

    callbacks_list = [checkpoint]

    model.fit(network_input, network_output, epochs=10, batch_size=50, callbacks=callbacks_list)


In [21]:
def train_network(n):
    notes = n
    print('Glance @ "notes" variable used', notes[1:10])
    n_vocab = len(set(notes))
    print('n_vocab is', n_vocab)
    network_input, network_output = prepare_sequences(notes, n_vocab)
    model = create_network(network_input, n_vocab)
    train(model, network_input, network_output)
train_network(n)

Glance @ "notes" variable used ['E-4', 'E4', '4.8', '4.8', 'B3', '8.11', '9.1.4', 'B4', 'E-5']
n_vocab is 72
Sequence length (window) used 5
Dict created note->int {'1': 0, '1.4': 1, '1.6': 2, '11': 3, '11.1.3': 4, '11.3': 5, '11.4': 6, '3': 7, '3.6': 8, '3.8': 9, '3.9': 10, '4': 11, '4.6': 12, '4.8': 13, '6': 14, '6.8': 15, '6.8.11': 16, '6.9': 17, '8': 18, '8.1': 19, '8.11': 20, '8.9': 21, '9': 22, '9.1': 23, '9.1.4': 24, '9.11': 25, 'A1': 26, 'A2': 27, 'A3': 28, 'A4': 29, 'A5': 30, 'A6': 31, 'B-2': 32, 'B-4': 33, 'B-5': 34, 'B1': 35, 'B2': 36, 'B3': 37, 'B4': 38, 'B5': 39, 'B6': 40, 'C#2': 41, 'C#3': 42, 'C#4': 43, 'C#5': 44, 'C#6': 45, 'C#7': 46, 'E-3': 47, 'E-4': 48, 'E-5': 49, 'E-6': 50, 'E-7': 51, 'E1': 52, 'E2': 53, 'E3': 54, 'E4': 55, 'E5': 56, 'E6': 57, 'F#2': 58, 'F#3': 59, 'F#4': 60, 'F#5': 61, 'F#6': 62, 'F3': 63, 'F5': 64, 'G#1': 65, 'G#2': 66, 'G#3': 67, 'G#4': 68, 'G#5': 69, 'G#6': 70, 'G5': 71}
Glance @ input [[[0.66666667]
  [0.76388889]
  [0.18055556]
  [0.18055556]


# Generate notes

In [9]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.layers import BatchNormalization as BatchNorm

In [10]:
def prepare_sequences(notes, pitchnames, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    # map between notes and integers and back
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    sequence_length = 100
    network_input = []
    output = []
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    normalized_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    normalized_input = normalized_input / float(n_vocab)

    return (network_input, normalized_input)

In [11]:
def create_network(network_input, n_vocab):
    """ create the structure of the neural network """
    model = Sequential()
    model.add(LSTM(
        100,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        recurrent_dropout=0.7,
        return_sequences=True
    ))
    model.add(LSTM(100))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('sigmoid'))
    model.compile(loss='categorical_crossentropy', optimizer=RMSprop(learning_rate=0.00000013))
    
    # Load the weights to each node
    model.load_weights('/content/drive/My Drive/notebooks/music/weights-improvement-01-5.3044-bigger.hdf5')

    return model

In [12]:
def generate_notes(model, network_input, pitchnames, n_vocab):
    """ Generate notes from the neural network based on a sequence of notes """
    # pick a random sequence from the input as a starting point for the prediction
    start = np.random.randint(0, len(network_input)-1)

    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))

    pattern = network_input[start]
    prediction_output = []

    # generate 500 notes
    for note_index in range(500):
        prediction_input = np.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(n_vocab)

        prediction = model.predict(prediction_input, verbose=0)

        index = np.argmax(prediction)
        #index = np.argmax(prediction)[0]

        result = int_to_note[index]
        prediction_output.append(result)

        pattern.append(index)
        pattern = pattern[1:len(pattern)]

    return prediction_output

In [13]:
def create_midi(prediction_output):
    """ convert the output from the prediction to notes and create a midi file
        from the notes """
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for pattern in prediction_output:
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # increase offset each iteration so that notes do not stack
        offset += 0.5

    midi_stream = stream.Stream(output_notes)
    #midi_stream.write('midi', fp='/content/drive/My Drive/notebooks/music/test_507pm_Weds.mid')
    return midi_stream

In [14]:
def generate(notes):
    """ Generate a piano midi file """
    #load the notes used to train the model
    notes = notes

    # Get all pitch names
    pitchnames = sorted(set(item for item in notes))
    # Get all pitch names
    n_vocab = len(set(notes))

    network_input, normalized_input = prepare_sequences(notes, pitchnames, n_vocab)
    model = create_network(normalized_input, n_vocab)
    prediction_output = generate_notes(model, network_input, pitchnames, n_vocab)
    midi_stream = create_midi(prediction_output)
    return midi_stream

In [15]:
midi_stream = generate(n)

In [16]:
from google.colab import files
files.download(midi_stream.write('midi', fp='/content/drive/My Drive/notebooks/music/LSTM_839am_Wed.mid'))

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>